In [6]:
import json
import pandas as pd
import numpy as np
import numexpr as ne
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
from dotenv import load_dotenv
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.cuda.amp as amp

from src.model.model import save_model, load_inference_model
from src.dataset.df import check_dataset
from src.utils.common import set_seed

from src.experiment.inference import inference
from src.experiment.initialize import init_exp
from src.utils.metrics import compute_surface_dice_score_from_volume
from src.model.model import build_model
from src.model.loss import get_lossfn
from src.utils.metrics import get_metrics
from src.model.scheduler import get_scheduler
from src.dataset.common import get_train_dataset

In [8]:
class cfg:
    debug = False
    check_dataset = False

    # = data CFG ====================================================

    dataset_path = "/kaggle/working/dataset/train01_xy_256_128_z_1_2/"
    train_dataset = "Base2dDataset"
    negative_sample_rate = 0.2

    # = experiment CFG =================================================

    project = "SenNet"
    exp_name = os.path.basename(os.getcwd())
    notes = "preliminary model"

    # = model CFG ======================================================

    model_arch = "Unet"
    backbone = "se_resnext50_32x4d"
    in_chans = 1
    target_size = 1

    # = training CFG ===================================================

    epochs = 60

    train_batch_size = 64
    valid_batch_size = train_batch_size

    loss = "DiceLoss"
    metrics = "Dice"
    lr = 5e-4
    num_workers = 12

    # = augmentation ===================================================

    image_size = 256
    train_aug = [
        A.RandomRotate90(p=0.5),
        A.RandomGamma(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        ToTensorV2(transpose_mask=True),
    ]

    valid_aug = [
        ToTensorV2(transpose_mask=True),
    ]

    # =============== inference ========================================

    test_dataset = "BaseInferenceDataset"
    stride = image_size // 2
    drop_egde_pixel = 32


load_dotenv("/kaggle/key.env")
set_seed()

In [9]:
def filter_dataset(df):
    # trainのうちlabelが全くないものは90%の確率で除外
    df["random"] = np.random.rand(len(df))
    df = df[(df["sum"] > 0) | (df["fold0"] == "valid") | (df["random"] < cfg.negative_sample_rate)]
    df = df.reset_index(drop=True)
    df = df.drop(["random"], axis=1)
    return df


df = pd.read_csv(f"{cfg.dataset_path}/dataset.csv")
df = filter_dataset(df)
# df = df.sample(1000).reset_index(drop=True)
if cfg.debug:
    df = df.sample(10000).reset_index(drop=True)
display(df)

if cfg.check_dataset:
    check_dataset(df, cfg)

C:\Users\welsh\AppData\Local\Temp\ipykernel_10256\4046888049.py:10: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{cfg.dataset_path}/dataset.csv")


,image_path,label_path,fname,kidney,x,y,z,std,sum,fold0,fold1,fold2
0,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x0_y0_z0_std0039_sum0,kidney_1_dense,0,0,0,39,0,valid,train,train
1,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x0_y0_z100_std0037_sum0,kidney_1_dense,0,0,100,37,0,valid,train,train
2,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x0_y0_z102_std0036_sum0,kidney_1_dense,0,0,102,36,0,valid,train,train
3,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x0_y0_z104_std0036_sum0,kidney_1_dense,0,0,104,36,0,valid,train,train
4,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x0_y0_z106_std0036_sum0,kidney_1_dense,0,0,106,36,0,valid,train,train
...,...,...,...,...,...,...,...,...,...,...,...,...
481727,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x896_y256_z990_std0261_sum1352,kidney_3_sparse,896,256,990,261,1352,train,NaN,NaN
481728,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x896_y256_z992_std0262_sum1473,kidney_3_sparse,896,256,992,262,1473,train,NaN,NaN
481729,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x896_y256_z994_std0263_sum1554,kidney_3_sparse,896,256,994,263,1554,train,NaN,NaN
481730,/kaggle/working/dataset/cropped_xy_256_128_z_1...,/kaggle/working/dataset/cropped_xy_256_128_z_1...,x896_y256_z996_std0264_sum1656,kidney_3_sparse,896,256,996,264,1656,train,NaN,NaN


In [4]:
if cfg.debug:
    print("!!!Debug mode!!!\n")
    cfg.epochs = 5

for fold in [2]:
    dataset = get_train_dataset(cfg)

    train_df = df[df[f"fold{fold}"] == "train"]
    valid_df = df[df[f"fold{fold}"] == "valid"]

    train_dataset = dataset(train_df, cfg, is_train=True)
    valid_dataset = dataset(valid_df, cfg, is_train=False)

    train_dataloader = DataLoader(train_dataset, batch_size=cfg.train_batch_size, num_workers=cfg.num_workers, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=cfg.valid_batch_size, num_workers=cfg.num_workers, shuffle=False)

    model = build_model(cfg.model_arch, cfg.backbone, cfg.in_chans, cfg.target_size)
    scaler = torch.cuda.amp.GradScaler()
    criterion = get_lossfn(cfg)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr)
    scheduler = get_scheduler(cfg, optimizer)
    metrics = get_metrics(cfg)

    # model, scaler, criterion, optimizer, scheduler, metrics = init_model(cfg)
    slacknotify = init_exp(fold, cfg)

    path_best = f"./{cfg.exp_name}/{cfg.exp_name}_best_fold{fold}.pth"
    path_last = f"./{cfg.exp_name}/{cfg.exp_name}_last_fold{fold}.pth"

    best_loss = float("inf")
    for epoch in range(cfg.epochs):
        model.train()
        total_loss = 0.0
        pbar_train = tqdm(enumerate(train_dataloader), total=len(train_dataloader), bar_format="{l_bar}{bar:10}{r_bar}{bar:-0b}")

        for i, (images, masks) in pbar_train:
            images, masks = images.cuda(), masks.cuda()
            optimizer.zero_grad()

            with amp.autocast():
                preds = model(images)
                loss = criterion(preds, masks)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.detach().item()

            loss_ = total_loss / (i + 1)
            lr = f"LR : {scheduler.get_lr()[0]:.2E}"
            gpu_mem = f"Mem : {torch.cuda.memory_reserved() / 1E9:.3g}GB"
            pbar_train.set_description(("%10s  " * 3 + "%10s") % (f"Epoch {epoch}/{cfg.epochs}", gpu_mem, lr, f"Loss: {loss_:.4f}"))

        train_loss = loss_
        scheduler.step()
        wandb.log({"epoch": epoch, "train_loss": train_loss})

        model.eval()
        total_loss = 0.0
        pbar_val = tqdm(enumerate(valid_dataloader), total=len(valid_dataloader), bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}")

        for i, (images, masks) in pbar_val:
            images, masks = images.cuda(), masks.cuda()
            with torch.no_grad():
                preds = model(images)
                loss = criterion(preds, masks)
                total_loss += loss.item()

            loss_ = total_loss / (i + 1)
            pbar_val.set_description(("%10s") % (f"Val Loss: {loss_:.4f}"))
        valid_loss = loss_
        wandb.log({"epoch": epoch, "valid_loss": valid_loss})

        if valid_loss < best_loss:
            print(f"loss : {valid_loss:.4f}\tSAVED MODEL\n")
            slacknotify.send_reply(f"epoch : {epoch}\tscore : {valid_loss:.4f}\tBEST")
            best_loss = valid_loss
            save_model(model, cfg, path_best, loss=loss)
        else:
            print(f"loss : {valid_loss:.4f}\n")
            slacknotify.send_reply(f"epoch : {epoch}\tscore : {valid_loss:.4f}")

    save_model(model, cfg, path_last, loss=valid_loss)
    wandb.config.update({"last_loss": valid_loss, "best_loss": best_loss})

    slacknotify.send_reply(f"{cfg.exp_name}_fold{fold} training finished\nbest loss : {best_loss:.4f} last loss : {loss_:.4f}", True)

    if wandb.run:
        wandb.finish()

model_arch:  Unet
backbone:  se_resnext50_32x4d


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: welshonionman. Use `wandb login --relogin` to force relogin


Epoch 0/60  Mem : 8.95GB  LR : 5.00E-04  Loss: 0.1188: 100%|██████████| 4352/4352 [13:36<00:00,  5.33it/s]
Val Loss: 0.2281: 100%|██████████| 2503/2503 [04:37<00:00,  9.03it/s]


loss : 0.2281	SAVED MODEL



Epoch 1/60  Mem : 9.95GB  LR : 5.00E-03  Loss: 0.1245: 100%|██████████| 4352/4352 [13:13<00:00,  5.48it/s]
Val Loss: 0.2013: 100%|██████████| 2503/2503 [04:34<00:00,  9.11it/s]


...

Epoch 58/60  Mem : 9.95GB  LR : 5.56E-06  Loss: 0.0530: 100%|██████████| 4352/4352 [13:10<00:00,  5.50it/s]
Val Loss: 0.1788: 100%|██████████| 2503/2503 [04:06<00:00, 10.13it/s]


loss : 0.1788



Epoch 59/60  Mem : 9.95GB  LR : 3.18E-06  Loss: 0.0530: 100%|██████████| 4352/4352 [13:11<00:00,  5.50it/s]
Val Loss: 0.1761: 100%|██████████| 2503/2503 [04:07<00:00, 10.13it/s]


loss : 0.1761



epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,▇█▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▆▂▂▂▁▂▂▃▃▁▂▂▃▄▁▂▂▃▅▂▃▁▂▁▂▃▃▃▂▂▂▃▃▃▄▄▄▄▃
epoch,59
train_loss,0.05297
valid_loss,0.17611


In [4]:
# valid
fold_dict = json.load(open("/kaggle/src/dataset/fold.json", "r"))
pth_type = "best"

for fold in [2]:
    kidney = fold_dict[f"fold{fold}"]["valid"][0]
    pth_path = f"./{cfg.exp_name}/{cfg.exp_name}_{pth_type}_fold{fold}.pth"
    stack_path = f"/kaggle/working/dataset/stack_train01/{kidney}_images.npy"
    label_path = f"/kaggle/working/dataset/stack_train01/{kidney}_labels.npy"
    preds_path = f"./preds/{kidney}_{pth_type}_preds.npy"

    model = load_inference_model(pth_path, cfg)

    print(kidney)
    print(stack_path)
    # inference(model, stack_path, preds_path, cfg)

    label = np.load(label_path)
    preds = np.load(preds_path)
    thresh_score_dict = {}

    for thresh in np.arange(0.1, 0.99, 0.1):
        thresh = round(thresh, 5)
        thresh_score_dict[thresh] = compute_surface_dice_score_from_volume(ne.evaluate("preds > thresh"), label)

    max_score_thresh = max(thresh_score_dict, key=thresh_score_dict.get)

    for thresh in np.arange(max_score_thresh - 0.1, max_score_thresh + 0.1, 0.01):
        thresh = round(thresh, 5)
        thresh_score_dict[thresh] = compute_surface_dice_score_from_volume(ne.evaluate("preds > thresh"), label)

    print(max(thresh_score_dict.items(), key=lambda x: x[1]))

model_name Unet
backbone se_resnext50_32x4d
fold_2
kidney_2_sparse
/kaggle/working/dataset/stack_clipped/kidney_2_sparse_images.npy
(0.67, 0.8495)


In [3]:
# infer
pth_type = "best"
fold = 2
kidneys = ["kidney_2_sparse", "kidney_3_sparse", "kidney_9_pseudo"]

pth_path = f"./{cfg.exp_name}/{cfg.exp_name}_{pth_type}_fold{fold}.pth"
for kidney in kidneys:
    stack_path = f"/kaggle/working/dataset/stack_train01/{kidney}_images.npy"
    preds_path = f"./preds/{kidney}_{pth_type}_preds.npy"

    model = load_inference_model(pth_path, cfg)

    print(stack_path)
    inference(model, stack_path, preds_path, cfg)

model_name Unet
backbone se_resnext50_32x4d
/kaggle/working/dataset/stack_clipped/kidney_2_sparse_images.npy
././preds/kidney_2_sparse_preds_0.npy


100%|██████████| 2216/2216 [10:13<00:00,  3.61it/s]


././preds/kidney_2_sparse_preds_1.npy


100%|██████████| 1040/1040 [09:36<00:00,  1.80it/s]


././preds/kidney_2_sparse_preds_2.npy


...

100%|██████████| 2140/2140 [10:50<00:00,  3.29it/s]


././preds/kidney_9_pseudo_preds_1.npy


100%|██████████| 1107/1107 [10:19<00:00,  1.79it/s]


././preds/kidney_9_pseudo_preds_2.npy


100%|██████████| 1643/1643 [10:39<00:00,  2.57it/s]


In [15]:
pth_type = "best"
kidneys = ["kidney_2_sparse", "kidney_3_sparse", "kidney_9_pseudo"]
for kidney in kidneys:
    stack_path = f"/kaggle/working/dataset/stack_train01/{kidney}_images.npy"
    preds_path = f"./preds/{kidney}_{pth_type}_preds.npy"
    preds_th_path = f"./preds/{kidney}_{pth_type}_preds_th.npy"
    preds = np.load(preds_path)
    preds_th = (preds > 0.67).astype(np.bool_)
    np.save(preds_th_path, preds > 0.67)